# B"H

## Setup 

In [1]:
!pip install nnfs

In [2]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

## Network Classes

### Dense layer

In [3]:
class Layer_Dense:

    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

### ReLU activation

In [4]:
class Activation_ReLU:

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Since we need to modify original variable,
        # let's make a copy of values first
        self.dinputs = dvalues.copy()

        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

### Softmax activation

In [5]:
# Softmax activation
class Activation_Softmax:

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs

        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1,
                                            keepdims=True))
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1,
                                            keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):

        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in \
                enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - \
                              np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix,
                                         single_dvalues)

### SGD optimizer

In [6]:
class Optimizer_SGD:

    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):

        # If we use momentum
        if self.momentum:

            # If layer does not contain momentum arrays, create them
            # filled with zeros
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                # If there is no momentum array for weights
                # The array doesn't exist for biases yet either.
                layer.bias_momentums = np.zeros_like(layer.biases)

            # Build weight updates with momentum - take previous
            # updates multiplied by retain factor and update with
            # current gradients
            weight_updates = \
                self.momentum * layer.weight_momentums - \
                self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            # Build bias updates
            bias_updates = \
                self.momentum * layer.bias_momentums - \
                self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        # Vanilla SGD updates (as before momentum update)
        else:
            weight_updates = -self.current_learning_rate * \
                             layer.dweights
            bias_updates = -self.current_learning_rate * \
                           layer.dbiases

        # Update weights and biases using either
        # vanilla or momentum updates
        layer.weights += weight_updates
        layer.biases += bias_updates


    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

### Adagrad optimizer

In [7]:
class Optimizer_Adagrad:

    # Initialize optimizer - set settings
    def __init__(self, learning_rate=1., decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):

        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2

        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * \
                         layer.dweights / \
                         (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * \
                        layer.dbiases / \
                        (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

### RMSprop optimizer


In [8]:
class Optimizer_RMSprop:

    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7,
                 rho=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):

        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update cache with squared current gradients
        layer.weight_cache = self.rho * layer.weight_cache + \
            (1 - self.rho) * layer.dweights**2
        layer.bias_cache = self.rho * layer.bias_cache + \
            (1 - self.rho) * layer.dbiases**2

        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * \
                         layer.dweights / \
                         (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * \
                        layer.dbiases / \
                        (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

### Adam optimizer

In [9]:
class Optimizer_Adam:

    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7,
                 beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):

        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update momentum  with current gradients
        layer.weight_momentums = self.beta_1 * \
                                 layer.weight_momentums + \
                                 (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * \
                               layer.bias_momentums + \
                               (1 - self.beta_1) * layer.dbiases
        # Get corrected momentum
        # self.iteration is 0 at first pass
        # and we need to start with 1 here
        weight_momentums_corrected = layer.weight_momentums / \
            (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / \
            (1 - self.beta_1 ** (self.iterations + 1))
        # Update cache with squared current gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + \
            (1 - self.beta_2) * layer.dweights**2

        layer.bias_cache = self.beta_2 * layer.bias_cache + \
            (1 - self.beta_2) * layer.dbiases**2
        # Get corrected cache
        weight_cache_corrected = layer.weight_cache / \
            (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / \
            (1 - self.beta_2 ** (self.iterations + 1))

        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * \
                         weight_momentums_corrected / \
                         (np.sqrt(weight_cache_corrected) +
                             self.epsilon)
        layer.biases += -self.current_learning_rate * \
                         bias_momentums_corrected / \
                         (np.sqrt(bias_cache_corrected) +
                             self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

### Common loss class

In [10]:
class Loss:

    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):

        # Calculate sample losses
        sample_losses = self.forward(output, y)

        # Calculate mean loss
        data_loss = np.mean(sample_losses)

        # Return loss
        return data_loss

### Cross-entropy loss

In [11]:
class Loss_CategoricalCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):

        # Number of samples in a batch
        samples = len(y_pred)
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]

        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples

### Combined Softmax activation and cross-entropy loss 

In [12]:
# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy():

    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)

        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)

        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples

## Scenario 1 - Vanilla SGD (LR 1)

### Dataset

In [18]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

### Model Architecture

In [19]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output 
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD()

### Training Loop

Remember, each full pass through all of the training data is called an **epoch**.

In [20]:
# Train in loop
for epoch in range(10001):

    # -- -------------------------------
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)

    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # -- -------------------------------
    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    
    # -- -------------------------------
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)


epoch: 0, acc: 0.263, loss: 1.099
epoch: 100, acc: 0.420, loss: 1.075
epoch: 200, acc: 0.440, loss: 1.062
epoch: 300, acc: 0.447, loss: 1.059
epoch: 400, acc: 0.453, loss: 1.057
epoch: 500, acc: 0.450, loss: 1.055
epoch: 600, acc: 0.453, loss: 1.052
epoch: 700, acc: 0.460, loss: 1.047
epoch: 800, acc: 0.453, loss: 1.050
epoch: 900, acc: 0.453, loss: 1.047
epoch: 1000, acc: 0.437, loss: 1.051
epoch: 1100, acc: 0.447, loss: 1.041
epoch: 1200, acc: 0.453, loss: 1.034
epoch: 1300, acc: 0.463, loss: 1.023
epoch: 1400, acc: 0.460, loss: 1.013
epoch: 1500, acc: 0.450, loss: 1.004
epoch: 1600, acc: 0.437, loss: 0.991
epoch: 1700, acc: 0.440, loss: 0.974
epoch: 1800, acc: 0.517, loss: 0.959
epoch: 1900, acc: 0.473, loss: 0.978
epoch: 2000, acc: 0.497, loss: 0.948
epoch: 2100, acc: 0.503, loss: 0.920
epoch: 2200, acc: 0.507, loss: 0.925
epoch: 2300, acc: 0.587, loss: 0.887
epoch: 2400, acc: 0.540, loss: 0.868
epoch: 2500, acc: 0.613, loss: 0.849
epoch: 2600, acc: 0.593, loss: 0.824
epoch: 2700, 

## Scenario 2 - SGD (LR 0.85)

### Dataset

In [24]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

### Model Architecture

In [25]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output 
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(learning_rate=.85)

### Training Loop

Remember, each full pass through all of the training data is called an **epoch**.

In [26]:
# Train in loop
for epoch in range(10001):

    # -- -------------------------------
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)

    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # -- -------------------------------
    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    
    # -- -------------------------------
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)


epoch: 0, acc: 0.290, loss: 1.099
epoch: 100, acc: 0.413, loss: 1.064
epoch: 200, acc: 0.443, loss: 1.060
epoch: 300, acc: 0.447, loss: 1.057
epoch: 400, acc: 0.460, loss: 1.054
epoch: 500, acc: 0.473, loss: 1.050
epoch: 600, acc: 0.487, loss: 1.047
epoch: 700, acc: 0.493, loss: 1.042
epoch: 800, acc: 0.473, loss: 1.035
epoch: 900, acc: 0.403, loss: 1.037
epoch: 1000, acc: 0.393, loss: 1.033
epoch: 1100, acc: 0.397, loss: 1.025
epoch: 1200, acc: 0.407, loss: 1.015
epoch: 1300, acc: 0.460, loss: 1.020
epoch: 1400, acc: 0.463, loss: 1.001
epoch: 1500, acc: 0.497, loss: 1.007
epoch: 1600, acc: 0.443, loss: 1.008
epoch: 1700, acc: 0.447, loss: 0.981
epoch: 1800, acc: 0.443, loss: 0.971
epoch: 1900, acc: 0.457, loss: 0.964
epoch: 2000, acc: 0.480, loss: 0.959
epoch: 2100, acc: 0.473, loss: 0.968
epoch: 2200, acc: 0.467, loss: 0.959
epoch: 2300, acc: 0.503, loss: 0.961
epoch: 2400, acc: 0.497, loss: 0.957
epoch: 2500, acc: 0.467, loss: 0.971
epoch: 2600, acc: 0.490, loss: 0.952
epoch: 2700, 

## Scenario 3 - SGD (LR 1, decay=1e-2)

[Vid](https://www.youtube.com/watch?v=TgP-i5A1sjY)

This model definitely got stuck, and the reason is almost certainly because the **learning rate decayed far too quickly** and became too small, **trapping the model in some local minimum**. 

This is most likely why, rather than wiggling, our accuracy and loss stopped changing at all.

### Dataset

In [27]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

### Model Architecture

In [28]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output 
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(decay=1e-2)

### Training Loop

Remember, each full pass through all of the training data is called an **epoch**.

In [29]:
# Train in loop
for epoch in range(10001):

    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)

    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


epoch: 0, acc: 0.293, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.417, loss: 1.084, lr: 0.5025125628140703
epoch: 200, acc: 0.447, loss: 1.066, lr: 0.33444816053511706
epoch: 300, acc: 0.443, loss: 1.063, lr: 0.2506265664160401
epoch: 400, acc: 0.447, loss: 1.061, lr: 0.2004008016032064
epoch: 500, acc: 0.457, loss: 1.060, lr: 0.1669449081803005
epoch: 600, acc: 0.457, loss: 1.060, lr: 0.14306151645207438
epoch: 700, acc: 0.453, loss: 1.060, lr: 0.1251564455569462
epoch: 800, acc: 0.450, loss: 1.059, lr: 0.11123470522803114
epoch: 900, acc: 0.447, loss: 1.059, lr: 0.10010010010010009
epoch: 1000, acc: 0.447, loss: 1.059, lr: 0.09099181073703366
epoch: 1100, acc: 0.443, loss: 1.059, lr: 0.08340283569641367
epoch: 1200, acc: 0.437, loss: 1.059, lr: 0.07698229407236336
epoch: 1300, acc: 0.433, loss: 1.059, lr: 0.07147962830593281
epoch: 1400, acc: 0.433, loss: 1.059, lr: 0.066711140760507
epoch: 1500, acc: 0.427, loss: 1.059, lr: 0.06253908692933083
epoch: 1600, acc: 0.427, loss: 1.059, lr: 0

## Scenario 4 - SGD (LR 1, decay=1e-3)


Better results than scenario 3.

But it still should be possible to find parameters that will give us even better results. For example, you may suspect that the initial learning rate is too high. It can make for a great exercise to attempt to find better settings. Feel free to try!

### Dataset

In [30]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

### Model Architecture

In [31]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output 
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(decay=1e-3)

### Training Loop

Remember, each full pass through all of the training data is called an **epoch**.

In [32]:
# Train in loop
for epoch in range(10001):

    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)

    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


epoch: 0, acc: 0.257, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.393, loss: 1.085, lr: 0.9099181073703367
epoch: 200, acc: 0.383, loss: 1.069, lr: 0.8340283569641367
epoch: 300, acc: 0.390, loss: 1.068, lr: 0.7698229407236336
epoch: 400, acc: 0.387, loss: 1.067, lr: 0.7147962830593281
epoch: 500, acc: 0.393, loss: 1.067, lr: 0.66711140760507
epoch: 600, acc: 0.390, loss: 1.066, lr: 0.6253908692933083
epoch: 700, acc: 0.390, loss: 1.066, lr: 0.5885815185403178
epoch: 800, acc: 0.393, loss: 1.065, lr: 0.5558643690939411
epoch: 900, acc: 0.390, loss: 1.063, lr: 0.526592943654555
epoch: 1000, acc: 0.397, loss: 1.061, lr: 0.5002501250625312
epoch: 1100, acc: 0.400, loss: 1.058, lr: 0.4764173415912339
epoch: 1200, acc: 0.413, loss: 1.054, lr: 0.45475216007276037
epoch: 1300, acc: 0.423, loss: 1.050, lr: 0.43497172683775553
epoch: 1400, acc: 0.427, loss: 1.044, lr: 0.4168403501458941
epoch: 1500, acc: 0.440, loss: 1.039, lr: 0.4001600640256102
epoch: 1600, acc: 0.437, loss: 1.032, lr: 0.38476337

## Scenario 5 - SGD (LR 1, decay=1e-3, momentum=0.9)

Great result!



### Dataset

In [36]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

### Model Architecture

In [37]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output 
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(decay=1e-3, momentum=0.9)

### Training Loop

Remember, each full pass through all of the training data is called an **epoch**.

In [38]:
# Train in loop
for epoch in range(10001):

    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)

    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


epoch: 0, acc: 0.350, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.487, loss: 1.026, lr: 0.9099181073703367
epoch: 200, acc: 0.513, loss: 0.931, lr: 0.8340283569641367
epoch: 300, acc: 0.597, loss: 0.802, lr: 0.7698229407236336
epoch: 400, acc: 0.687, loss: 0.704, lr: 0.7147962830593281
epoch: 500, acc: 0.767, loss: 0.563, lr: 0.66711140760507
epoch: 600, acc: 0.847, loss: 0.461, lr: 0.6253908692933083
epoch: 700, acc: 0.867, loss: 0.411, lr: 0.5885815185403178
epoch: 800, acc: 0.637, loss: 0.730, lr: 0.5558643690939411
epoch: 900, acc: 0.880, loss: 0.366, lr: 0.526592943654555
epoch: 1000, acc: 0.880, loss: 0.352, lr: 0.5002501250625312
epoch: 1100, acc: 0.833, loss: 0.440, lr: 0.4764173415912339
epoch: 1200, acc: 0.890, loss: 0.325, lr: 0.45475216007276037
epoch: 1300, acc: 0.870, loss: 0.323, lr: 0.43497172683775553
epoch: 1400, acc: 0.873, loss: 0.325, lr: 0.4168403501458941
epoch: 1500, acc: 0.880, loss: 0.307, lr: 0.4001600640256102
epoch: 1600, acc: 0.887, loss: 0.299, lr: 0.38476337

## Scenario ?? - Adam

### Dataset

In [15]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

### Model Architecture

In [14]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-7)

### Training Loop

Remember, each full pass through all of the training data is called an **epoch**.

In [16]:
# Train in loop
for epoch in range(10001):

    # -- ------------------------------
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, y)

    # -- ------------------------------
    # Get some metrics    
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # -- ------------------------------    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # -- ------------------------------
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.323, loss: 1.099, lr: 0.05
epoch: 100, acc: 0.590, loss: 0.861, lr: 0.04999752512250644
epoch: 200, acc: 0.693, loss: 0.725, lr: 0.04999502549496326
epoch: 300, acc: 0.763, loss: 0.659, lr: 0.049992526117345455
epoch: 400, acc: 0.790, loss: 0.601, lr: 0.04999002698961558
epoch: 500, acc: 0.827, loss: 0.556, lr: 0.049987528111736124
epoch: 600, acc: 0.807, loss: 0.532, lr: 0.049985029483669646
epoch: 700, acc: 0.827, loss: 0.506, lr: 0.049982531105378675
epoch: 800, acc: 0.813, loss: 0.493, lr: 0.04998003297682575
epoch: 900, acc: 0.830, loss: 0.470, lr: 0.049977535097973466
epoch: 1000, acc: 0.847, loss: 0.444, lr: 0.049975037468784345
epoch: 1100, acc: 0.820, loss: 0.447, lr: 0.049972540089220974
epoch: 1200, acc: 0.837, loss: 0.411, lr: 0.04997004295924593
epoch: 1300, acc: 0.843, loss: 0.408, lr: 0.04996754607882181
epoch: 1400, acc: 0.813, loss: 0.416, lr: 0.049965049447911185
epoch: 1500, acc: 0.830, loss: 0.410, lr: 0.04996255306647668
epoch: 1600, acc: 0.857, lo